In [432]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns


# Load Data

In [433]:
book_path = r"preprocessed_data\book_processed.csv"
rating_path = r"preprocessed_data\rating_processed.csv"
user_data = r"preprocessed_data\user_processed.csv"

In [434]:
book_df = pd.read_csv(book_path)
book_df.drop(columns=["Unnamed: 0"], inplace=True)
book_df.sort_values(by=["ISBN"], inplace=True)
book_df.reset_index(drop=True, inplace=True)

rating_df = pd.read_csv(rating_path)
rating_df.drop(columns=["Unnamed: 0"], inplace=True)
rating_df.sort_values(by=["User-ID", "ISBN"], inplace=True)
rating_df.reset_index(drop=True, inplace=True)

user_df = pd.read_csv(user_data)
user_df.drop(columns=["Unnamed: 0"], inplace=True)
user_df.sort_values(by=["User-ID"], inplace=True)
user_df.reset_index(drop=True, inplace=True)

print("Books")
display(book_df.head(3))
print("Ratings")
display(rating_df.head(3))
print("Users")
display(user_df.head(3))

Books


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0000913154,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator),1967.0,Simon &amp; Schuster
1,0001010565,Mog's Christmas,Judith Kerr,1992.0,Collins
2,0001046438,Liar,Stephen Fry,NaN,Harpercollins Uk


Ratings


,User-ID,ISBN,Book-Rating
0,2,0195153448,0
1,8,0002005018,5
2,8,0060973129,0


Users


,User-ID,Location,Age
0,1,"nyc, new york, usa",34
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",34


# Data Preparation

## include book cover images

In [435]:
from IPython.core.display_functions import display
temp_df = pd.read_csv("dataset\Books.csv")
select_col = ["ISBN", "Image-URL-L"]
temp_df = temp_df[select_col]

book_df = pd.merge(book_df, temp_df, on="ISBN", how="left")

print("Books")
display(book_df.head(3))


C:\Users\Jimmy\AppData\Local\Temp\ipykernel_16884\4259117308.py:2: DtypeWarning: Columns (0: Year-Of-Publication) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv("dataset\Books.csv")


Books


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0000913154,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator),1967.0,Simon &amp; Schuster,http://images.amazon.com/images/P/0000913154.0...
1,0001010565,Mog's Christmas,Judith Kerr,1992.0,Collins,http://images.amazon.com/images/P/0001010565.0...
2,0001046438,Liar,Stephen Fry,NaN,Harpercollins Uk,http://images.amazon.com/images/P/0001046438.0...


## create seperate author and publisher table

In [436]:
author_df = pd.DataFrame()
author_df["name"] = book_df["Book-Author"].unique()
author_df["author_id"] = author_df.index
author_df = author_df[["author_id", "name"]]

publisher_df = pd.DataFrame()
publisher_df["name"] = book_df["Publisher"].unique()
publisher_df["publisher_id"] = publisher_df.index
publisher_df = publisher_df[["publisher_id", "name"]]

print("Authors")
display(author_df.head(3))
print("Publishers")
display(publisher_df.head(3))

Authors


,author_id,name
0,0,C. van Amerongen (translator)
1,1,Judith Kerr
2,2,Stephen Fry


Publishers


,publisher_id,name
0,0,Simon &amp; Schuster
1,1,Collins
2,2,Harpercollins Uk


## replace author, and publisher with id on books table

In [437]:
book_df = pd.merge(book_df,author_df, left_on="Book-Author", right_on="name", how="left")
book_df.drop(columns=["Book-Author","name"], inplace=True)

book_df = pd.merge(book_df,publisher_df, left_on="Publisher", right_on="name", how="left")
book_df.drop(columns=["Publisher","name"], inplace=True)
print("Books")
display(book_df.head(3))

Books


,ISBN,Book-Title,Year-Of-Publication,Image-URL-L,author_id,publisher_id
0,0000913154,The Way Things Work: An Illustrated Encycloped...,1967.0,http://images.amazon.com/images/P/0000913154.0...,0,0
1,0001010565,Mog's Christmas,1992.0,http://images.amazon.com/images/P/0001010565.0...,1,1
2,0001046438,Liar,NaN,http://images.amazon.com/images/P/0001046438.0...,2,2


## create new book_id then drop the old one (ISBN) on books & ratings tables 

In [438]:
book_df.sort_values("ISBN", inplace=True)
book_df.reset_index(inplace=True)
book_df.rename(columns={"index":"book_id"}, inplace=True)

rating_df = pd.merge(rating_df, book_df[["ISBN", "book_id"]], on="ISBN", how="left")
rating_df.drop(columns="ISBN", inplace=True)

book_df.drop(columns="ISBN", inplace=True)

print("Books")
display(book_df.head(3))

print("Ratings")
display(rating_df.head(3))

Books


,book_id,Book-Title,Year-Of-Publication,Image-URL-L,author_id,publisher_id
0,0,The Way Things Work: An Illustrated Encycloped...,1967.0,http://images.amazon.com/images/P/0000913154.0...,0,0
1,1,Mog's Christmas,1992.0,http://images.amazon.com/images/P/0001010565.0...,1,1
2,2,Liar,NaN,http://images.amazon.com/images/P/0001046438.0...,2,2


Ratings


,User-ID,Book-Rating,book_id
0,2,0,25028
1,8,5,73
2,8,0,8211


## use birth year instead of age on Users table

In [439]:
import datetime
user_df["birth_year"] = datetime.datetime.now().year-user_df["Age"]
user_df.drop(columns="Age", inplace=True)

## rename columns in all tables

In [440]:
new_columns_map = {"Book-Title" : "name",
                   "Year-Of-Publication" : "published_year",
                   "Image-URL-L" : "image_url"}
book_df.rename(columns=new_columns_map, inplace=True)

new_columns_map = {"Book-Rating" : "book_rating",
                   "User-ID": "user_id"}
rating_df.rename(columns=new_columns_map, inplace=True)

new_columns_map = {"User-ID" : "user_id",
                   "Location" : "location"}
user_df.rename(columns=new_columns_map, inplace=True)

## show all tables

In [441]:
all_df = {"Books" : book_df, 
          "Ratings" : rating_df, 
          "Users" : user_df, 
          "Authors" : author_df, 
          "Publishers" : publisher_df}
for name, df in all_df.items():
    print(name)
    display(df.head(3))

Books


,book_id,name,published_year,image_url,author_id,publisher_id
0,0,The Way Things Work: An Illustrated Encycloped...,1967.0,http://images.amazon.com/images/P/0000913154.0...,0,0
1,1,Mog's Christmas,1992.0,http://images.amazon.com/images/P/0001010565.0...,1,1
2,2,Liar,NaN,http://images.amazon.com/images/P/0001046438.0...,2,2


Ratings


,user_id,book_rating,book_id
0,2,0,25028
1,8,5,73
2,8,0,8211


Users


,user_id,location,birth_year
0,1,"nyc, new york, usa",1992
1,2,"stockton, california, usa",2008
2,3,"moscow, yukon territory, russia",1992


Authors


,author_id,name
0,0,C. van Amerongen (translator)
1,1,Judith Kerr
2,2,Stephen Fry


Publishers


,publisher_id,name
0,0,Simon &amp; Schuster
1,1,Collins
2,2,Harpercollins Uk


# Data Trasformation

## add new fields to Users table

In [442]:
import numpy as np
user_df["username"] = pd.NA
user_df["email"] = pd.NA
user_df["user_image"] = pd.NA
user_df = user_df[["user_id", "username", "birth_year", "email", "location", "user_image"]]

# add new user with username and email
new_user = [user_df.user_id.max().item() + 1, "Jimmoi", "2004", "amonthep.se@kkumail.com", "Nongkrusri, kalasin, th", pd.NA]
user_df.loc[len(user_df)] = new_user
print("Users")
user_df.tail(3)

Users


,user_id,username,birth_year,email,location,user_image
278856,278857,<NA>,1992,<NA>,"knoxville, tennessee, usa",<NA>
278857,278858,<NA>,1992,<NA>,"dublin, n/a, ireland",<NA>
278858,278859,Jimmoi,2004,amonthep.se@kkumail.com,"Nongkrusri, kalasin, th",<NA>


## add new fields to ratings table

In [443]:
rating_df["comment"] = pd.NA
rating_df["is_favourite"] = False
rating_df = rating_df[["user_id", "book_id", "book_rating", "comment", "is_favourite"]]


# add some random favourite
mask = rating_df[rating_df["book_rating"]>8].sample(frac=0.2).index
rating_df.loc[mask,"is_favourite"] = True

# add some random review
mask = rating_df.sample(frac=0.5).index
rating_df.loc[mask,"comment"] = "This is a comment"

print("ratings")
rating_df.head()

ratings


,user_id,book_id,book_rating,comment,is_favourite
0,2,25028,0,<NA>,False
1,8,73,5,This is a comment,False
2,8,8211,0,This is a comment,False
3,8,60198,0,This is a comment,False
4,8,71711,0,This is a comment,False


## create new table user_wishlist

In [444]:
user_wishlist_df = user_df.sample(frac=0.005).merge(book_df.sample(frac=0.005), how='cross')
user_wishlist_df = user_wishlist_df[["user_id", "book_id"]]
user_wishlist_df = pd.merge(user_wishlist_df, rating_df, on=["user_id", "book_id"], how="left", indicator=True)
user_wishlist_df = user_wishlist_df[user_wishlist_df['_merge'] == 'left_only']
user_wishlist_df = user_wishlist_df[["user_id", "book_id"]]
user_wishlist_df["is_favourite"] = False
user_wishlist_df.loc[user_wishlist_df.sample(frac=0.5).index, "is_favourite"] = True

print("user_wishlist")
user_wishlist_df.sample(10)

user_wishlist


,user_id,book_id,is_favourite
505255,258049,250732,True
1312969,163274,189299,False
1138832,267341,140617,True
1624937,206701,182031,False
1200506,133639,24898,True
232604,82390,214223,False
1111416,134814,69111,True
629418,218337,81388,True
1751023,149412,59718,False
1836484,84069,36295,False


## rename id column

In [445]:
book_df.rename(columns={"book_id":"id"}, inplace = True)
author_df.rename(columns={"author_id":"id"}, inplace = True)
publisher_df.rename(columns={"publisher_id":"id"}, inplace = True)



In [446]:

all_df = {"Books" : book_df, 
          "user_reviews" : rating_df, 
          "Users" : user_df, 
          "Authors" : author_df, 
          "Publishers" : publisher_df,
          "user_wishlists" : user_wishlist_df}
for name, df in all_df.items():
    print(name)
    display(df.head(3))


Books


,id,name,published_year,image_url,author_id,publisher_id
0,0,The Way Things Work: An Illustrated Encycloped...,1967.0,http://images.amazon.com/images/P/0000913154.0...,0,0
1,1,Mog's Christmas,1992.0,http://images.amazon.com/images/P/0001010565.0...,1,1
2,2,Liar,NaN,http://images.amazon.com/images/P/0001046438.0...,2,2


user_reviews


,user_id,book_id,book_rating,comment,is_favourite
0,2,25028,0,<NA>,False
1,8,73,5,This is a comment,False
2,8,8211,0,This is a comment,False


Users


...

,id,name
0,0,C. van Amerongen (translator)
1,1,Judith Kerr
2,2,Stephen Fry


Publishers


,id,name
0,0,Simon &amp; Schuster
1,1,Collins
2,2,Harpercollins Uk


user_wishlists


,user_id,book_id,is_favourite
0,259896,160067,True
1,259896,144971,False
2,259896,203756,True


In [448]:
output_dir = Path("data_for_db")
for name, df in all_df.items():
    df.to_csv(output_dir/(name+".csv"), index=False)